<a href="https://colab.research.google.com/github/hselino/complaint_analysis/blob/main/translated_turna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content/drive/MyDrive/veriseti/


segmented_class.csv  translated1000.csv


In [ ]:
!pip install transformers datasets scikit-learn evaluate


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder


In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/veriseti/translated1000.csv")
df = df.drop(columns=["Unnamed: 0"], errors='ignore')
df = df.dropna()


In [ ]:
!ls /content/drive/MyDrive/


'Başlıksız Diyagram.drawio'
'Capstone Project Report IEEE Conference Paper Template.doc'
'Capstone Project Report Template.docx'
'--cevaplitestler.com-----------------------5.SINIF IŞIĞIN YAYILMASI TESTİ-1 (1).gdoc'
'--cevaplitestler.com-----------------------5.SINIF IŞIĞIN YAYILMASI TESTİ-1.gdoc'
 classification.ipynb
 cleaned_dataset.csv
'cleaned_dataset_no_suffixes kopyası.csv'
'Colab Notebooks'
 data_cleaning.py
 dataset.csv
 Figure_1.png
 koklerine_ayrilmis_dataset.csv.zip
 norm.py
 sikayet_siniflandirma_modeli.zip
 tokenization.py
 Untitled0.ipynb
 veriseti
'WhatsApp Görsel 2023-03-19 saat 22.01.38.jpg'
'WhatsApp Görsel 2023-03-19 saat 22.01.44.jpg'


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/veriseti/segmented_class.csv")
df = df.drop(columns=["Unnamed: 0"], errors='ignore')
df = df.dropna()

label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['category'])

# Label mapping ve num_labels
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
num_labels = len(label_mapping)

# Kontrol için yazdır
print(df.head())
print(f"Etiket Sayısı: {num_labels}")


NameError: name 'LabelEncoder' is not defined

In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
label_encoder = LabelEncoder()
df['encoded_label'] = label_encoder.fit_transform(df['category'])


In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EarlyStoppingCallback
)
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder


In [ ]:
print(df.head())
print(df['encoded_label'].value_counts())
print(f"Etiket Sayısı: {df['encoded_label'].nunique()}")


                                      complaint_text  \
0  kendi homepoint finansal olarak kısmen ol cidd...   
1  equifax kredi raporum sahtekarlık uyarım var r...   
2  son zaman aldatmaca işle para kimlik çal kişis...   
3  dağ zirves financial tan kredi ön plan çık ilg...   
4  arizona doğum tarih ss transunion llc tüket ci...   

                              category  encoded_label  
0         ödeme süreci sırasında sorun            127  
1       raporunuzun uygunsuz kullanımı            107  
2          borçlu borç alma denemeleri             17  
3  borçlu servic iletişime geçilemiyor             19  
4        raporunuzdaki yanlış bilgiler            106  
encoded_label
106    2347
107    1269
77      896
18      594
137     400
       ... 
50        1
100       1
44        1
31        1
94        1
Name: count, Length: 138, dtype: int64
Etiket Sayısı: 138


In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['complaint_text'].tolist(),
    df['encoded_label'].tolist(),
    test_size=0.2,
    random_state=42
)


In [ ]:
model_name = "boun-tabi-LMG/TURNA"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=df['encoded_label'].nunique())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/21.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.68k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.57G [00:00<?, ?B/s]

Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at boun-tabi-LMG/TURNA and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=128)


In [ ]:
class TurkishDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [ ]:
train_dataset = TurkishDataset(train_encodings, train_labels)
val_dataset = TurkishDataset(val_encodings, val_labels)


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = torch.argmax(torch.tensor(pred.predictions), axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


In [ ]:
training_args = TrainingArguments(
    output_dir="./results-turna",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,   # Küçük tuttuk
    per_device_eval_batch_size=2,    # Küçük tuttuk
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir='./logs-turna',
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


In [ ]:
print(type(train_dataset))
print(type(val_dataset))


<class '__main__.TurkishDataset'>
<class '__main__.TurkishDataset'>


In [ ]:
print(model)


T5ForSequenceClassification(
  (transformer): T5Model(
    (shared): Embedding(32128, 1024)
    (encoder): T5Stack(
      (embed_tokens): Embedding(32128, 1024)
      (block): ModuleList(
        (0): T5Block(
          (layer): ModuleList(
            (0): T5LayerSelfAttention(
              (SelfAttention): T5Attention(
                (q): Linear(in_features=1024, out_features=1024, bias=False)
                (k): Linear(in_features=1024, out_features=1024, bias=False)
                (v): Linear(in_features=1024, out_features=1024, bias=False)
                (o): Linear(in_features=1024, out_features=1024, bias=False)
                (relative_attention_bias): Embedding(32, 16)
              )
              (layer_norm): T5LayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (1): T5LayerFF(
              (DenseReluDense): T5DenseGatedActDense(
                (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
                (w

In [ ]:
print(training_args)


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.EPOCH,
eval_use_gather_object=False

In [ ]:
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


In [ ]:
from torch.utils.data import Dataset


In [ ]:

num_labels = df['encoded_label'].nunique()
print(f"num_labels: {num_labels}")


num_labels: 138


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(
    "dbmdz/bert-base-turkish-cased", num_labels=num_labels
)


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,   # RAM için küçük
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir='./logs',
    report_to="none",
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


In [ ]:
trainer.train()


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,2.741500,2.864210,0.300500,0.189766,0.188252,0.300500
2,2.553100,2.637514,0.356000,0.279765,0.242975,0.356000
3,2.230400,2.551604,0.362000,0.294544,0.284393,0.362000
4,2.009900,2.722615,0.356000,0.304094,0.290886,0.356000
5,1.817100,2.880269,0.377000,0.324899,0.321971,0.377000


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m

TrainOutput(global_step=20000, training_loss=2.381113864135742, metrics={'train_runtime': 802.7133, 'train_samples_per_second': 99.65, 'train_steps_per_second': 49.831, 'total_flos': 2633994083658240.0, 'train_loss': 2.381113864135742, 'epoch': 5.0})

In [ ]:
model_path = "/content/drive/MyDrive/veriseti/fine_tuned_model-turna"

model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)


('/content/drive/MyDrive/veriseti/fine_tuned_model-turna/tokenizer_config.json',
 '/content/drive/MyDrive/veriseti/fine_tuned_model-turna/special_tokens_map.json',
 '/content/drive/MyDrive/veriseti/fine_tuned_model-turna/tokenizer.json')

In [ ]:
def predict(texts):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    inputs = tokenizer(texts, return_tensors="pt", truncation=True, padding=True, max_length=128)
    inputs = {key: val.to(device) for key, val in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predictions = torch.argmax(probs, dim=-1)

    predictions = predictions.cpu().numpy()
    labels = label_encoder.inverse_transform(predictions)
    return labels


In [ ]:
sample_texts = [
    "ATM'den para çekerken hata oldu, hesabımdan para düştü ama para vermedi."
]

print(predict(sample_texts))


['hesabı yönetmek']


In [ ]:
short_text = "Kredi kartımdan bilgim dışında para çekildi. İadesini istiyorum."
medium_text = (
    "Geçen hafta internetten aldığım ürün hala kargoya verilmedi. Müşteri hizmetlerini aradım ama sürekli bekletiliyorum. "
    "Siparişim iptal edildi mi bilmiyorum. Ücret iadesi de yapılmadı."
)
long_text = (
    "28 Nisan'da cep telefonumu teknik servise verdim. Telefonumun ekranında piksel hatası vardı ve garanti kapsamındaydı. "
    "Ancak 15 gün geçmesine rağmen hiçbir geri dönüş alamadım. Müşteri hizmetlerini defalarca aradım, her seferinde farklı bilgi verildi. "
    "Bir temsilci yedek telefon temin edileceğini söyledi ama hala teslim edilmedi. Üstelik telefonun tamir süresi yasal olarak 20 iş gününü geçti. "
    "Bu durumdan çok mağdur oldum ve artık tüketici hakem heyetine başvurmayı düşünüyorum. Lütfen acil geri dönüş yapılsın."
)



In [ ]:
print("🔹 SHORT:")
print(predict([short_text]))

print("\n🔸 MEDIUM:")
print(predict([medium_text]))

print("\n🔻 LONG:")
print(predict([long_text]))


🔹 SHORT:
['kredi kartı almak']

🔸 MEDIUM:
['hesabı yönetmek']

🔻 LONG:
['hesabı yönetmek']


In [ ]:
text = (
    "Yaklaşık 2 ay önce yeni bir kredi kartı başvurusu yaptım ve kartım elime ulaştı. Ancak kartımı kullanmaya başladığım ilk günden itibaren, "
    "harcamalarımda yersiz işlem ücreti yansıtıldığını fark ettim. Müşteri hizmetlerini aradım ve bu ücretin otomatik olarak kesildiğini söylediler. "
    "Ancak başvuru sırasında böyle bir bilgilendirme yapılmamıştı. Kartımı iptal ettirmek istiyorum ama iptal sırasında da cezai işlem uygulanacağını belirtiyorlar. "
    "Bu mağduriyetin acilen giderilmesini talep ediyorum."
)

print(predict([text]))

['hesabı yönetmek']


In [ ]:
text = (
    "Geçtiğimiz ay bir trafik kazası geçirdim ve aracımın hasar tespiti için kasko şirketine başvurdum. Eksper raporu 10 gün sonra hazırlandı ve onarım için yetkili servise yönlendirildim. "
    "Ancak yetkili servis, eksper raporunda belirtilen bazı parçaların değişmesi gerektiğini söyledi. Şirket ise sadece onarım yapılacağını, parça değişimi olmayacağını belirtip kabul etmedi. "
    "Bu durum hem yasal haklarımı ihlal ediyor hem de güvenlik açısından risk oluşturuyor. Tüketici Hakem Heyetine başvurmadan önce çözüm bekliyorum."
)

print(predict([text]))

['raporunuzdaki yanlış bilgiler']


In [ ]:
text = (
    "Online alışveriş sitesinden 15 gün önce verdiğim sipariş hala kargoya verilmedi. Ürün açıklamasında 3 iş günü içinde kargolanacağı yazıyordu. "
    "Canlı destek ile görüştüğümde sistemsel bir hata olduğunu ve mağduriyetimin giderileceğini söylediler ama hala bir gelişme yok. "
    "Siparişi iptal etmek istediğimde ise ürün 'hazırlık aşamasında' olduğu için iptal edemiyorum. Bu sorunun çözülmesini ve mağduriyetimin giderilmesini istiyorum."
)

print(predict([text]))

['hesabı yönetmek']


In [ ]:
print(label_encoder.classes_)


['araç yeniden işlendi aracı satıldı' 'aşırı ücretler'
 'bakiye transfer ücreti' 'bakiye transferi'
 'banka hesabına yapılan ücretleri durduramıyorum'
 'banka hesabınızdan para çekme işlemlerini durduramıyorum'
 'başvurmadığınız kredi aldı' 'beklemediğiniz ücretler faiz'
 'beklenmedik diğer ücretler'
 'biriyle iletişim kurmak uygunsuz şekilde bilgi paylaşmakla tehdit edildi'
 'borcun ifşa doğrulaması' 'borç atm kartı kullanma'
 'borç hakkında yazılı bildirim' 'borç veren hizmetçimle uğraşmak'
 'borç veren hizmetçinizle uğraşmak' 'borç verenin okulum olduğu sorunu'
 'borç verenle iletişime geçemiyorum' 'borçlu borç alma denemeleri'
 'borçlu olmayan borcu tahsil etme girişimleri'
 'borçlu servic iletişime geçilemiyor' 'diğer' 'diğer hizmet sorunları'
 'diğer hizmet sorunu' 'diğer işlem problemi' 'diğer işlem sorunları'
 'diğer özellikler terimler sorunlar' 'diğer ücret'
 'dolandırıcılık uyarıları güvenlik donmaları ilgili sorun'
 'ek eklenti ürünleri hizmetleri ilgili sorun' 'elektronik 